In [1]:
import backtrader as bt
import datetime
import pandas as pd
import math

In [2]:
from lutils.stock import LTdxHq

In [3]:
ltdxhq = LTdxHq()

In [4]:
# ll = ltdxhq.stock_list()

In [5]:
# ll[(ll['pre_close'] > 30) & (ll['pre_close'] < 50)]

In [6]:
# 399300
code = '603900' # 000032 300142 603636 600519 688567
# df = ltdxhq.get_k_data_daily('300142') # 000032 300142 603636 600519
# df = ltdxhq.get_k_data_15min(code) #' # 000032 300142 603636 600519
# df = ltdxhq.to_qfq(code, df)

In [7]:
# df.index = df.index.unique(level=1)

In [8]:
df15 = ltdxhq.get_k_data_15min(code, qfq=True) #' # 000032 300142 603636 600519
df60 = ltdxhq.get_k_data_1hour(code, qfq=True) #' # 000032 300142 603636 600519
df1d = ltdxhq.get_k_data_daily(code, qfq=True) #' # 000032 300142 603636 600519

In [9]:
df15.index = df15.index.unique(level=1)
df60.index = df60.index.unique(level=1)
df1d.index = df1d.index.unique(level=0)

In [10]:
ltdxhq.close()

In [11]:
df15.index = pd.to_datetime(df15.index)
df60.index = pd.to_datetime(df60.index)
df1d.index = pd.to_datetime(df1d.index)

In [12]:
data15 = df15['2021-01-01':]
# data60 = df60['2021-01-01':]
# data1d = df1d['2021-01-01':]

In [13]:
class MultiTFStrategy(bt.Strategy):
    params = (
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 3.0),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
        
        ('short', 30),
        ('long', 70),
    )
    
    # states defination
    Empty, M15Hold, H1Hold, D1Hold = range(4)
    States = [
        'Empty', 'M15Hold', 'H1Hold', 'D1Hold',
    ]
    
    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):

        self.rsi15 = bt.indicators.RSI_EMA(self.dnames.hs15m.close, period=14) # RSI_SMA
        self.rsi60 = bt.indicators.RSI_EMA(self.dnames.hs1h.close, period=14)
        self.rsi1d = bt.indicators.RSI_EMA(self.dnames.hs1d.close, period=14)
    
        self.st = self.Empty
        self.st_map = {
            self.Empty: self._empty,
            self.M15Hold: self._m15hold,
            self.H1Hold: self._h1hold,
            self.D1Hold: self._d1hold,
        }
        
        # To keep track of pending orders
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )

            else:  # Sell
                self.log(
                    'SELL EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # just call state_map function
        self.order = self.st_map[self.st]()
        
        # Check if we are in the market and no buy order issued
        if self.position and not self.order:
            # Already in the market ... we might sell
            if self.rsi1d > self.params.long:
                self.st = self.Empty
                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
                
    def _empty(self):
        if self.rsi15 < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # 20% of the cash
            share = int(math.floor((0.2*cash)/price))

            # set state
            self.st = self.M15Hold
            return self.buy(size=share)
        
    def _m15hold(self):
        if self.rsi60 < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash ( 60% )
            share = int(math.floor((0.6*cash)/price))
            
            # set state
            self.st = self.H1Hold
            return self.buy(size=share)
        
    def _h1hold(self):
        if self.rsi1d < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash (80%)
            share = int(math.floor((0.8*cash)/price))
            
            # set state
            self.st = self.D1Hold
            return self.buy(size=share)
        
    def _d1hold(self):
        return None

In [14]:
cerebro = bt.Cerebro(oldtrades=True)

# feed15 = bt.feeds.PandasData(dataname=data15, openinterest=None, compression=15, timeframe=bt.TimeFrame.Minutes)
# feed60 = bt.feeds.PandasData(dataname=data60, openinterest=None, compression=60, timeframe=bt.TimeFrame.Minutes)
# feed1d = bt.feeds.PandasData(dataname=data1d, openinterest=None, compression=1, timeframe=bt.TimeFrame.Days)

# cerebro.adddata(feed15, name='hs15m')
# cerebro.adddata(feed60, name='hs1h')
# cerebro.adddata(feed1d, name='hs1d')

feed = bt.feeds.PandasData(dataname=data15, openinterest=None, compression=15, timeframe=bt.TimeFrame.Minutes)
cerebro.adddata(feed, name='hs15m')
cerebro.resampledata(feed, name='hs1h', timeframe=bt.TimeFrame.Minutes, compression=60)
cerebro.resampledata(feed, name='hs1d', timeframe=bt.TimeFrame.Days)

cerebro.addstrategy(MultiTFStrategy)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-01-25T10:00:00, BUY EXECUTED, St: M15Hold, Size: 349, Price: 5.74, Cost: 2003.26, Comm 1.00
2021-01-25T10:15:00, BUY EXECUTED, St: H1Hold, Size: 834, Price: 5.75, Cost: 4795.50, Comm 2.40
2021-01-25T10:30:00, BUY EXECUTED, St: D1Hold, Size: 438, Price: 5.83, Cost: 2553.54, Comm 1.28
2021-02-22T10:00:00, SELL EXECUTED, St: Empty, Size: -1621, Price: 6.27, Cost: 9352.30, Comm 5.08
2021-02-22T10:00:00, OPERATION PROFIT, GROSS 811.37, NET 801.61
2021-02-23T14:00:00, BUY EXECUTED, St: M15Hold, Size: 357, Price: 6.03, Cost: 2152.71, Comm 1.08
2021-02-23T14:15:00, SELL EXECUTED, St: Empty, Size: -357, Price: 6.03, Cost: 2152.71, Comm 1.08
2021-02-23T14:15:00, OPERATION PROFIT, GROSS 0.00, NET -2.15
2021-02-23T14:30:00, BUY EXECUTED, St: M15Hold, Size: 358, Price: 6.02, Cost: 2155.16, Comm 1.08
2021-02-23T14:45:00, SELL EXECUTED, St: Empty, Size: -358, Price: 6.03, Cost: 2155.16, Comm 1.08
2021-02-23T14:45:00, OPERATION PROFIT, GROSS 3.58, NET 1.42
2021-

In [15]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2021, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 640x480 with 11 Axes>]]